In [ ]:
!pip install langchain
!pip install langchain_community
!pip install sentence_transformers
!pip install unstructured
!pip install unstructured[pdf]
!pip install llama-cpp-python
!pip -q install peft chromadb
!pip install ctransformers

In [ ]:
!wget -O BioMistral-7B.Q5_K_M.gguf https://huggingface.co/MaziyarPanahi/BioMistral-7B-GGUF/resolve/main/BioMistral-7B.Q5_K_M.gguf?download=true

In [ ]:
!wget https://github.com/nandhishtr/DiseasePredictionLLM/archive/refs/heads/main.zip

In [ ]:
!unzip main.zip

In [ ]:
import os
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.llms import LlamaCpp
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts.few_shot import FewShotPromptTemplate

# initializes an embeddings model
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

print(embeddings)

# loader = TextLoader("health_report.txt")
loader = DirectoryLoader("/content/DiseasePredictionLLM-main/dataset_folder", glob="**/*.txt", show_progress=True)

docs = loader.load()

print(len(docs), "*******************")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=70)

pages = loader.load_and_split(text_splitter)

db = Chroma.from_documents(pages, embeddings, persist_directory='/content/db')

llm = CTransformers(model="MaziyarPanahi/BioMistral-7B-GGUF", model_file="BioMistral-7B.Q8_0.gguf", temperature=0.7,
                    max_tokens=2048, top_p=1, n_ctx=2048, config={'context_length': 1024})


In [7]:
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Please refer to the below examples to answer user query:

Example 1 :

question : Hello, there is a pain around the navel, I don’t know what's going on (female, 29 years old).. How long are you facing this? It has been there for three days.. Do you have any other symptoms? No symptoms. Have you used any medication? No.  Can you diagnose and recommend some precautionary measures?
answer : The pain in the navel area could be related to gastric dysfunction. Gastric dysfunction is a common symptom, which can be caused by food digestion problems. The symptoms of gastric dysfunction include abdominal pain, loss of appetite, loose stools or constipation, diarrhea, bloating, etc. In this case, it's best to drink enough water and eat light food. It is recommended not to drink tea in hot condition as well as avoid drinking soda. You should also limit the consumption of spicy food and alcoholic beverages.

Example 2 :

question : Hello, I have continuous head ache. How long are you facing this ? It has been there for over a week. Do you have any other symptoms? I have runny nose and cold. Have you used any medication? I have taken paracetamol. Can you diagnose and recommend some precautionary measures?
answer : Headache may have different causes and this one lasts for a week, so it is advisable to checkup for brain CT or MRI and in some cases do a lumbar puncture as well as blood tests.

Only return the helpful answer. Answer must be detailed and well explained.

Context: {context}
Question: {question}


Helpful answer:
"""
retriever = db.as_retriever(search_kwargs={"k": 1})
prompt = PromptTemplate(input_variables=['context', 'question'], template=prompt_template)

if llm is not None and db is not None:
  qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": prompt})
else:
    print("LLM or Vector Database not initialized")



print("Bot: Hello There, Welcome! I am a disease prediction bot, here to help you.\n"
      "Caution: I am only suggesting based on my current knowledge. In no way it is an alternative to consulting a doctor.")

# Define questions
questions = [
    "How may I help you today?",
    "How long are you facing this?",
    "Do you have any other symptoms?",
    "Have you used any medication?"
]

# Initialize total_input
print("Bot:", questions[0])
total_input = input("User: ") + ". "

# Loop through questions
for question in questions[1:]:
    print("Bot:", question)
    user_input = input("User: ")
    total_input += question + " " + user_input + ". "  # Concatenate user input with a period and space

main_question = "Can you diagnose and recommend some precautionary measures?"

final_question = total_input + " " + main_question
print(final_question)

result = qa_chain.invoke({"query": final_question})
print("Bot:", result["result"])

# # Ask if the user is satisfied
user_input = input("Bot: Are you satisfied with my answer? (Yes/No): ")

# If not satisfied, ask for additional information
if user_input.lower() == "no":
    print("Bot: Please provide me with some additional information to predict better.")
    additional_info = input("User: ")
    final_question += ". Additional information: " + additional_info + " " + main_question
    result = qa_chain.invoke({"query": final_question})
    print("Bot:", result["result"])

print("Bot: Thank you! Get well soon :)")

Bot: Hello There, Welcome! I am a disease prediction bot, here to help you.
Caution: I am only suggesting based on my current knowledge. In no way it is an alternative to consulting a doctor.
Bot: How may I help you today?
User: I am having pain around my navel
Bot: How long are you facing this?
User: for about a week
Bot: Do you have any other symptoms?
User: no other symptoms
Bot: Have you used any medication?
User: no
I am having pain around my navel. How long are you facing this? for about a week. Do you have any other symptoms? no other symptoms. Have you used any medication? no.  Can you diagnose and recommend some precautionary measures?
Bot: This type of pain is not common, especially in women. It may be caused by gastric dysfunction, which can occur due to food digestion problems. The symptoms of gastric dysfunction include abdominal pain, loss of appetite, loose stools or constipation, diarrhea, bloating, etc. In this case, it's best to drink enough water and eat light food. 